In [15]:
from AniList_extractor import AniListUser
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
df_completed, df_plan_to_watch = AniListUser('thems22').get_dfs()

In [29]:
df_plan_to_watch['english'] = np.where(df_plan_to_watch['english'].isna(), df_plan_to_watch['romaji'], df_plan_to_watch['english'])
df_plan_to_watch = df_plan_to_watch.dropna()

df_completed['english'] = np.where(df_completed['english'].isna(), df_completed['romaji'], df_completed['english'])

In [30]:
df_completed['watched'] = np.int8(1)
df_plan_to_watch['watched'] = np.int8(0)

/tmp/ipykernel_964/3464976671.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_plan_to_watch['watched'] = np.int8(0)


In [94]:
df_watched = pd.concat([df_completed[['description', 'watched', 'english']], df_plan_to_watch[['description', 'watched', 'english']]])
df_watched = df_watched.reset_index()

In [95]:
df_watched['description'] = df_watched['description'].str.replace('<.*?>', '', regex=True).str.replace('\n', '')

In [96]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')

In [97]:
tfidf_matrix = tf.fit_transform(df_watched['description'])

In [98]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [99]:
max_idx = df_watched[df_watched['watched']==1].index[-1]

In [100]:
plan_to_watch_cosine = cosine_sim[max_idx::, max_idx::].shape

In [101]:
df_plan_to_watch.shape

(301, 242)

In [102]:
titles = df_watched['english']
indices = pd.Series(df_watched.index, index=df_watched['english'])

In [160]:
def get_recommendations(title, df):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores
    movie_indices = [i[0] for i in sim_scores]
    df['rank'] = pd.DataFrame(list(enumerate(movie_indices))).set_index(1)
    df = df[df['watched']==0]
    return df.sort_values(by='rank')['english'][0:10]

In [165]:
df_watched[df_watched['english'].str.contains('mn', case=False)]

,index,description,watched,english,rank
149,149,Rin Asogi is an immortal private detective wit...,1,RIN ~Daughters of Mnemosyne~,45
479,220,Why would an overworked woman want to be summo...,0,The Saint's Magic Power is Omnipotent,314


In [174]:
df_completed[df_completed['score']==9]['english']

5                                       Yona of the Dawn
24                             My Hero Academia Season 3
26                                          Chainsaw Man
27                                           Chihayafuru
29                                         Chihayafuru 3
33                                              Claymore
39                      The Mystic Archives of Dantalian
45                                          Death Parade
47                                             Dr. STONE
54                                         Durarara!! X2
57                         Fate/Grand Order: First Order
72     Legend of the Galactic Heroes: Die Neue These ...
75                                          Golden Kamuy
76                                 Golden Kamuy Season 2
77                                 Golden Kamuy Season 3
83                                              Hellsing
87                                   High School DxD New
88                             

In [180]:
get_recommendations("Death Parade", df_watched)

284                                         Bartender
509                                        Soul Eater
497                                       Ghost Hound
362    Eden of The East the Movie I: The King of Eden
365                                     Hi Score Girl
371                           Hozuki's Coolheadedness
361                                  Eden of The East
309                   Daily Lives of High School Boys
458                                  Outbreak Company
400                         Teasing Master Takagi-san
Name: english, dtype: object